In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
inputs = keras.Input(shape=(784,))
# img_inputs = keras.Input(shape=(32, 32, 3))

In [11]:
inputs.shape

TensorShape([None, 784])

In [12]:
inputs.dtype

tf.float32

In [14]:
dense = layers.Dense(64, activation='relu', name='layer1')
x = dense(inputs)

In [15]:
x = layers.Dense(64, activation="relu" , name="layer2")(x)
outputs = layers.Dense(10)(x)


In [16]:
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")


In [17]:
model.summary()


Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 layer1 (Dense)              (None, 64)                50240     
                                                                 
 layer2 (Dense)              (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64,
                    epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])


Epoch 1/2
750/750 [==============================] - 7s 5ms/step - loss: 0.3403 - accuracy: 0.9020 - val_loss: 0.1753 - val_accuracy: 0.9506
Epoch 2/2
750/750 [==============================] - 3s 4ms/step - loss: 0.1551 - accuracy: 0.9530 - val_loss: 0.1302 - val_accuracy: 0.9623
313/313 - 1s - loss: 0.1254 - accuracy: 0.9630 - 746ms/epoch - 2ms/step
Test loss: 0.12541651725769043
Test accuracy: 0.9629999995231628


In [21]:
model.save("path_to_my_model")
del model
# Recreate the exact same model purely from the file:
model = keras.models.load_model("path_to_my_model")


INFO:tensorflow:Assets written to: path_to_my_model\assets


In [ ]:
encoder_input = keras.Input(shape=(28, 28, 1), name="img")
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation="relu")(x)

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
autoencoder.summary()


In [26]:
def get_model():
    inputs = keras.Input(shape=(128,))
    outputs = layers.Dense(1)(inputs)
    return keras.Model(inputs, outputs)


model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = keras.Input(shape=(128,))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
num_tags = 12  # Number of unique issue tags
num_words = 10000  # Size of vocabulary obtained when preprocessing text data
num_departments = 4  # Number of departments for predictions

title_input = keras.Input(shape=(None,), name="title")  # Variable-length sequence of ints
# Variable-length sequence of ints
body_input = keras.Input(shape=(None,), name="body")
tags_input = keras.Input(shape=(num_tags,), name="tags")  # Binary vectors of size `num_tags`

# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input)
# Embed each word in the text into a 64-dimensional vector
body_features = layers.Embedding(num_words, 64)(body_input)

# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = layers.LSTM(128)(title_features)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
body_features = layers.LSTM(32)(body_features)

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input])

# Stick a logistic regression for priority prediction on top of the features
priority_pred = layers.Dense(1, name="priority")(x)
# Stick a department classifier on top of the features
department_pred = layers.Dense(num_departments, name="department")(x)

# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred],
)


In [27]:
vgg19 = tf.keras.applications.VGG19()


574726144/574710816 [==============================] - 18s 0us/step


In [28]:
features_list = [layer.output for layer in vgg19.layers]


In [29]:
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)

img = np.random.random((1, 224, 224, 3)).astype("float32")
extracted_features = feat_extraction_model(img)


In [ ]:
units = 32
timesteps = 10
input_dim = 5

# Define a Functional model
inputs = keras.Input((None, units))
x = layers.GlobalAveragePooling1D()(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


class CustomRNN(layers.Layer):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.units = units
        self.projection_1 = layers.Dense(units=units, activation="tanh")
        self.projection_2 = layers.Dense(units=units, activation="tanh")
        # Our previously-defined Functional model
        self.classifier = model

    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.units))
        for t in range(inputs.shape[1]):
            x = inputs[:, t, :]
            h = self.projection_1(x)
            y = h + self.projection_2(state)
            state = y
            outputs.append(y)
        features = tf.stack(outputs, axis=1)
        print(features.shape)
        return self.classifier(features)


rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, timesteps, input_dim)))


In [30]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [31]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]


In [32]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


In [33]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)


Fit model on training data
Epoch 1/2
782/782 [==============================] - 5s 5ms/step - loss: 0.3447 - sparse_categorical_accuracy: 0.9026 - val_loss: 0.1739 - val_sparse_categorical_accuracy: 0.9512
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.1618 - sparse_categorical_accuracy: 0.9524 - val_loss: 0.1475 - val_sparse_categorical_accuracy: 0.9560


In [34]:
history.history

{'loss': [0.34474995732307434, 0.16175459325313568],
 'sparse_categorical_accuracy': [0.9025800228118896, 0.9523599743843079],
 'val_loss': [0.1739272028207779, 0.14747333526611328],
 'val_sparse_categorical_accuracy': [0.951200008392334, 0.9559999704360962]}

In [35]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Evaluate on test data
79/79 [==============================] - 0s 4ms/step - loss: 0.1627 - sparse_categorical_accuracy: 0.9505
test loss, test acc: [0.16273784637451172, 0.9505000114440918]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [36]:
predictions

array([[6.73837121e-07, 3.40380986e-08, 7.31289911e-05, 3.60348466e-04,
        1.99594972e-08, 8.20781395e-07, 2.41558523e-10, 9.99527216e-01,
        2.50784638e-07, 3.75285490e-05],
       [1.66525160e-06, 5.05111639e-05, 9.89058793e-01, 1.04709538e-02,
        1.25099253e-09, 3.28389928e-04, 1.86778179e-05, 1.25029778e-10,
        7.10008826e-05, 1.07781428e-09],
       [3.69893951e-06, 9.96383548e-01, 8.03223345e-04, 2.95626873e-04,
        1.11248984e-04, 2.00452443e-04, 6.86029307e-05, 1.04444753e-03,
        1.05688255e-03, 3.22760970e-05]], dtype=float32)

In [42]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model


In [43]:
import tensorflow as tf
from tensorflow import data
from tensorflow import keras
from tensorflow.data import Dataset


model = get_compiled_model()

# First, let's create a training Dataset instance.
# For the sake of our example, we'll use the same MNIST data as before.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))


Epoch 1/3
782/782 [==============================] - 4s 4ms/step - loss: 0.3317 - sparse_categorical_accuracy: 0.9050
Epoch 2/3
782/782 [==============================] - 4s 5ms/step - loss: 0.1590 - sparse_categorical_accuracy: 0.9537
Epoch 3/3
782/782 [==============================] - 4s 5ms/step - loss: 0.1190 - sparse_categorical_accuracy: 0.9655
Evaluate
157/157 [==============================] - 1s 3ms/step - loss: 0.1168 - sparse_categorical_accuracy: 0.9649


{'loss': 0.11675912886857986, 'sparse_categorical_accuracy': 0.964900016784668}

In [45]:
x_train.shape

(50000, 784)